In [66]:
import pandas as pd
import numpy as np
import tweepy
import requests
import re
import time
from tweepy import OAuthHandler

In [67]:
# Twitter App access keys for @user
#my app access keys

# Consume:
CONSUMER_KEY    = "uBLfQkRIFDpEp2P1apLvidqbl"
CONSUMER_SECRET = "vtTvcPhegI5xdexVDQk4tpvcwTskSfaPxXFl8DienAHAUJ7lTS"

# Access:
ACCESS_TOKEN  = "921917165950906368-UMBLKE7q5LAGxfIKMJZzlbqCxZfs2Ed"
ACCESS_SECRET = "1ZDrPKAf5N8G8BIZyH3SOXXmRYz78nnt59ijfExO6fWk5"



In [68]:
# We import our access keys:
from credentials import *    # This will allow us to use the keys as variables

# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth,wait_on_rate_limit=True)
    return api


In [69]:
# We create an extractor object:
extractor = twitter_setup()

In [70]:
t = pd.read_csv('RachaelMaskell_cleaned_tweets.csv')

t['id'][0]

872580595297267712

In [71]:

#获得的是
def retweet_users_of_a_tweet(tweet_id):
    retweets = extractor.retweets(tweet_id, 100)
    return [rt.user.id for rt in retweets]


def t_all_tweets(user,n):
    result = []
    count = 0
    for x in range(n):
        tweets = extractor.user_timeline(id=user,count=200,page=x+1,include_rts=True)
        result += tweets
        count += 1
        if (x+1)%10 == 0:
            print ('sleep for 90 seconds')
            time.sleep(90)
        print (count, 'of ', n, 'pages done')
    return result

#t_mentions返回的是user （提到目标的次数和转推次数的和）
def t_mentions(user):
    tweets = t_all_tweets(user, 2) # first 2 pages timeline, 16 pages max
    #这里用了t_all_tweets来获取前两页的tweets。2页是400条
    t_text = ''
    for t in tweets:
        t_text += t.text
    return len(re.findall('@bennison_andrew', t_text)) # number of direct mentions + retweets
#这里要把提到的和转推的tweets 都储存下来 进行下一步分析 加上

#计算influence=follower*mention
def t_user_rank(users):
    udic = {}
    count = 0
    for user in users:
        screen_name = extractor.get_user(id=user).screen_name
        follower = extractor.get_user(id=user).followers_count
        mention = t_mentions(user)
        #这里用了t_mention来计算次数
        udic[screen_name] = [follower, mention, (follower*mention)]
        count += 1
        print (count, 'of', len(users), 'users added into dictionary')
        if count%5 == 0:
            print ('sleep for one minute')
            time.sleep(60)
    return udic


def t_tweets_influencers(n):
    count = 0
    for i in range(n):
        if not i:
            udic = t_user_rank(retweet_users_of_a_tweet(t['id'][i])) # first 3 users, 100 max
            #{names[i]:d.values()[i] for i in range(len(names))} in python2.x
            #v = list(d.values())
            #{names[i]:v[i] for i in range(len(names))} in python 3
           # v_follower = list (udic.values()[0])
            length= len(list(udic.values()))
            print(type(length))
            
            set_list= list(udic.values())
            while i < range(length):
                follower = set_list[i][0]
            
            follower = [ list(udic.values()[x])[0] for x in range(length)]
            mention = [list(udic.values()[x])[1] for x in range(length)]
            score = [list(udic.values()[x])[2] for x in range(length)]
            keys = udic.keys()
            t_id = [t['id'][i] for x in range(len(udic))]
            newdic = {'t_id':t_id,'influencer':keys,'score':score,'mention':mention,'follower':follower}
        else:
            udic = t_user_rank(retweet_users_of_a_tweet(t['id'][i])) # first 3 users, 100 max
            follower = [list(udic.values()[x])[0] for x in range(length)]
            mention = [list(udic.values()[x])[1] for x in range(length)]
            score = [list(udic.values()[x])[2] for x in range(length)]
            keys = udic.keys()
            t_id = [t['id'][i] for x in range(len(udic))]
            #.update({'item3': 3})
            newdic.update({'t_id':t_id})
           # newdic['t_id'].update( += t_id
            newdic.update({'influencer': keys})
            #newdic['influencer'] += keys
            newdic.update({'score':score })
            #newdic['score'] += score
            newdic.update({'mention':mention })
            #newdic['mention'] += mention
            newdic.update({'follower':follower })
            #newdic['follower'] += follower
        count += 1
        print ('-------', count, 'of', n, 'tweets analyzed', '-------')
    return newdic

result = t_tweets_influencers(2) # first 2 popular tweets, 100 max
df = pd.DataFrame(result)
print(df)
#df.to_csv('influencers(10 posts).csv', encoding='utf-8')

print ('project is done!')

1 of  2 pages done
2 of  2 pages done
1 of 45 users added into dictionary
1 of  2 pages done
2 of  2 pages done
2 of 45 users added into dictionary
1 of  2 pages done
2 of  2 pages done
3 of 45 users added into dictionary
1 of  2 pages done
2 of  2 pages done
4 of 45 users added into dictionary
1 of  2 pages done
2 of  2 pages done
5 of 45 users added into dictionary
sleep for one minute
1 of  2 pages done
2 of  2 pages done
6 of 45 users added into dictionary
1 of  2 pages done
2 of  2 pages done
7 of 45 users added into dictionary
1 of  2 pages done
2 of  2 pages done
8 of 45 users added into dictionary
1 of  2 pages done
2 of  2 pages done
9 of 45 users added into dictionary
1 of  2 pages done
2 of  2 pages done
10 of 45 users added into dictionary
sleep for one minute
1 of  2 pages done
2 of  2 pages done
11 of 45 users added into dictionary
1 of  2 pages done
2 of  2 pages done
12 of 45 users added into dictionary
1 of  2 pages done
2 of  2 pages done
13 of 45 users added into dic

TypeError: 'dict_values' object does not support indexing

In [73]:
print(t_user_rank(users))

NameError: name 'users' is not defined